# Custom cnn 

In [1]:
%%bash
source env1/bin/activate

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

## Appunti - Custom Estimator
```python
# input will be: [[1,2,3],[2,10,3],...] and this layer will 
# output [[0.2,0.3...], len([...]) = EMBEDDING_SIZE]
tf.contrib.layers.embed_sequence(  
    ids: # [batch_size, doc_length] Tensor of type int32 or int64 with symbol ids.  
    vocab_size: # Integer number of symbols in vocabulary.  
    embed_dim: # Integer number of dimensions for embedding matrix.  
    initializer: # An initializer for the embeddings, if None default for current scope is used.  
    )  
```
-----------------------------
```python
# finestra di lunghezza kernel_size che si sposta all'interno di ogni vettore in input
conv = tf.layers.conv1d(
    inputs= # Tensor input
    filters= # the dimensionality of the output space (i.e. the number of filters in the convolution).
    kernel_size= # An integer or tuple/list of a single integer, specifying the length of the 1D convolution window.
    padding= # One of "valid" or "same" (case-insensitive).
    activation= # function. Set it to None to maintain a linear activation.
)
```
--------------------
```python
# riduco la dimensione dell'uscita di conv prendendo il valore max per ogni vettore
pool = tf.reduce_max(input_tensor=conv, axis=1) 
```
--------------------
```python
# ottengo un tensore con un label per riga, tutto su una sola colonna
# my alternative: labels = tf.reshape(labels, [-1, 1]) if mode != tf.estimator.ModeKeys.PREDICT else labels
labels = tf.reshape(labels, [-1, 1])
```
----------------------
### head
```python
# Given logits (or output of a hidden layer), a Head knows how to compute predictions, loss, default metric and export signature
head = tf.contrib.estimator.binary_classification_head() # Creates a _Head for single label binary classification
output = head.create_estimator_spec(
    features=features,
    labels=labels,
    mode=mode,
    logits=logits, 
    train_op_fn=_train_op_fn)
```
------------------------
### my_model_fn(features, labels, mode)
```python
def my_model_fn(
   features, # This is batch_features from input_fn
   labels,   # This is batch_labels from input_fn
   mode,     # Instance of tf.estimator.ModeKeys, see below
   params):  # {} facoltativo
    
# Se uso params allora alla creazione
# params = {...}
# my_class = tf.estimator.Estimator(model_fn=my_model_fn,
#                                         model_dir=...,
#                                         params=params)
 
```

In [6]:
features = [[1,2,3],[4,5,6]] # frase con parole sostituite da indici

vocabulary = {'cat':1, 'is':2} # vocabolario con tutte le parole e i relativi indici

EMBEDDING_SIZE = 10 # 'cat' -> len(embedding('cat'))

head = tf.contrib.estimator.binary_classification_head() # Creates a _Head for single label binary classification

# Creates a _Head for single label binary classification
head = tf.contrib.estimator.binary_classification_head() 


In [10]:
def cnn_model_fn(features, labels, mode, params):    
    
    # INPUT
    input_layer = tf.contrib.layers.embed_sequence(
        features['x'], len(vocabulary), EMBEDDING_SIZE,   # posso fare features['x'] perchè prima ho fatto dataset = dataset.map(parser)
        initializer=params['embedding_initializer'])
     
    # DROPOUT LAYER
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout_emb = tf.layers.dropout(inputs=input_layer, 
                                    rate=0.2, 
                                    training=training) # se non sono in training niente dropout
    
    
    # CNN
    conv = tf.layers.conv1d(
        inputs=dropout_emb,
        filters=32,
        kernel_size=3,
        padding="same",
        activation=tf.nn.relu)
    
    # Global Max Pooling
    pool = tf.reduce_max(input_tensor=conv, axis=1)
    
    
    # Full connected layer + dropout
    hidden = tf.layers.dense(inputs=pool, units=250, activation=tf.nn.relu)
    dropout_hidden = tf.layers.dropout(inputs=hidden, 
                                       rate=0.2, 
                                       training=training)
    
    # Output-logits layer
    logits = tf.layers.dense(inputs=dropout_hidden, units=1)
    
    
    # Metto il label in (una) colonna
    if labels is not None: # This will be None when predicting
        labels = tf.reshape(labels, [-1, 1])
        
    
    # Gradient descend optimizator
    optimizer = tf.train.AdamOptimizer()
    
    
    # Usata nell'output
    def _train_op_fn(loss):
        return optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    
    
    # Output
    # head = tf.contrib.estimator.binary_classification_head()
    output = head.create_estimator_spec(
        features=features,
        labels=labels,
        mode=mode,
        logits=logits, 
        train_op_fn=_train_op_fn)
    
    return output